## Preprocessing

In [ ]:
# Import our dependencies

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
import tensorflow as tf

application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


## Optimization 1 Drop Also Status, seems redundant with the target

In [ ]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df = application_df.drop(columns=["EIN","NAME",'STATUS'])
application_df.head(2)

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1-9999,N,108590,1


In [ ]:
# Determine the number of unique values in each column.
application_df.nunique()

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [ ]:
# Look at APPLICATION_TYPE value counts for binning
application_df['APPLICATION_TYPE'].value_counts()


T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

## Optimization 2 increase the value for bin to 737 ocurrances

In [ ]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`

application_types_to_replace = application_df['APPLICATION_TYPE'].value_counts()[application_df['APPLICATION_TYPE'].value_counts()<737].index

application_types_to_replace



Index(['T7', 'T10', 'T9', 'T13', 'T12', 'T2', 'T25', 'T14', 'T29', 'T15',
       'T17'],
      dtype='object')

In [ ]:

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()

T3       27037
T4        1542
Other     1529
T6        1216
T5        1173
T19       1065
T8         737
Name: APPLICATION_TYPE, dtype: int64

In [ ]:
# Look at CLASSIFICATION value counts for binning
application_df['CLASSIFICATION'].value_counts()


C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [ ]:
# You may find it helpful to look at CLASSIFICATION value counts >1
application_df['CLASSIFICATION'].value_counts()[application_df['CLASSIFICATION'].value_counts()>1]


C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1238       10
C1278       10
C1235        9
C1237        9
C7210        7
C2400        6
C1720        6
C4100        6
C1257        5
C1600        5
C1260        3
C2710        3
C0           3
C3200        2
C1234        2
C1246        2
C1267        2
C1256        2
Name: CLASSIFICATION, dtype: int64

## Optimization 3 decrase the vale for bin to 777 ocurrances

In [ ]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`

classifications_to_replace = application_df['CLASSIFICATION'].value_counts()[application_df['CLASSIFICATION'].value_counts()<777].index

classifications_to_replace


Index(['C1700', 'C4000', 'C5000', 'C1270', 'C2700', 'C2800', 'C7100', 'C1300',
       'C1280', 'C1230', 'C1400', 'C7200', 'C2300', 'C1240', 'C8000', 'C7120',
       'C1500', 'C1800', 'C6000', 'C1250', 'C8200', 'C1238', 'C1278', 'C1235',
       'C1237', 'C7210', 'C2400', 'C1720', 'C4100', 'C1257', 'C1600', 'C1260',
       'C2710', 'C0', 'C3200', 'C1234', 'C1246', 'C1267', 'C1256', 'C2190',
       'C4200', 'C2600', 'C5200', 'C1370', 'C1248', 'C6100', 'C1820', 'C1900',
       'C1236', 'C3700', 'C2570', 'C1580', 'C1245', 'C2500', 'C1570', 'C1283',
       'C2380', 'C1732', 'C1728', 'C2170', 'C4120', 'C8210', 'C2561', 'C4500',
       'C2150'],
      dtype='object')

In [ ]:

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")

# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
Other     1484
C7000      777
Name: CLASSIFICATION, dtype: int64

In [ ]:
# Recheck the number of unique values in each column.
application_df.nunique()

APPLICATION_TYPE             7
AFFILIATION                  6
CLASSIFICATION               7
USE_CASE                     5
ORGANIZATION                 4
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [ ]:
# Separate the target and the features variables
y = application_df["IS_SUCCESSFUL"]
X = application_df.drop(columns="IS_SUCCESSFUL")

In [ ]:
# Confirm that the features are not labeled with the target
X

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT
0,Other,Independent,C1000,ProductDev,Association,0,N,5000
1,T3,Independent,C2000,Preservation,Co-operative,1-9999,N,108590
2,T5,CompanySponsored,C3000,ProductDev,Association,0,N,5000
3,T3,CompanySponsored,C2000,Preservation,Trust,10000-24999,N,6692
4,T3,Independent,C1000,Heathcare,Trust,100000-499999,N,142590
...,...,...,...,...,...,...,...,...
34294,T4,Independent,C1000,ProductDev,Association,0,N,5000
34295,T4,CompanySponsored,C3000,ProductDev,Association,0,N,5000
34296,T3,CompanySponsored,C2000,Preservation,Association,0,N,5000
34297,T5,Independent,C3000,ProductDev,Association,0,N,5000


In [ ]:
# Separate the numerical and categorical columns in the Features DataFrame,
# store the names of the columns so they can be used in the Column Transformer later on

numerical_columns = X.select_dtypes(include=['int','float64']).columns
categorical_columns = X.select_dtypes(include=['object','boolean']).columns

print(numerical_columns)
print(categorical_columns)

Index(['ASK_AMT'], dtype='object')
Index(['APPLICATION_TYPE', 'AFFILIATION', 'CLASSIFICATION', 'USE_CASE',
       'ORGANIZATION', 'INCOME_AMT', 'SPECIAL_CONSIDERATIONS'],
      dtype='object')


In [ ]:
# Configure the preprocessor with different transformations dependig of the data type

preprocessor = ColumnTransformer(
    transformers=[
        ('nominal', OneHotEncoder(handle_unknown='ignore'), categorical_columns),
        ('numeric', StandardScaler(), numerical_columns)
        ])

In [ ]:
# Split the data into train and test datasets

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=33)

In [ ]:
# Apply preprocessor to the data

X_train = preprocessor.fit_transform(X_train).toarray()
X_test  = preprocessor.transform(X_test).toarray()

In [ ]:
# Check the transformed data
pd.DataFrame(X_train)

,0,1,2,3,4,5,6,7,8,9,...,31,32,33,34,35,36,37,38,39,40
0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,-0.032255
1,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,-0.032255
2,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,-0.032255
3,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,-0.032255
4,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,-0.032255
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27434,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,-0.032255
27435,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,-0.032255
27436,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,-0.032255
27437,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,-0.031901


In [ ]:
# Get the shape of the training data
X_train.shape

(27439, 41)

## Compile, Train and Evaluate the Model

### Optimization 4 Increase the number of neurons add more hidden layer and try with other activation functions

In [ ]:
# Define the model
model = tf.keras.models.Sequential([
  tf.keras.layers.Dense(units=120, activation='tanh', input_shape=(X_train.shape[1],)),
  tf.keras.layers.Dense(units=100, activation='tanh'),
   tf.keras.layers.Dense(units=80, activation='tanh'),
  tf.keras.layers.Dense(units=1, activation='sigmoid')
])

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Model summary
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 120)               5040      
                                                                 
 dense_1 (Dense)             (None, 100)               12100     
                                                                 
 dense_2 (Dense)             (None, 80)                8080      
                                                                 
 dense_3 (Dense)             (None, 1)                 81        
                                                                 
Total params: 25301 (98.83 KB)
Trainable params: 25301 (98.83 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


## Optimization 3 create a callback to save the weights for the best epoch, this will be used later on

In [ ]:

checkpoint_path = "/chekpoint/training_4.ckpt"


# Create a callback that saves the model's weights
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=0,
                                                 mode ='max',
                                                 monitor ='accuracy',
                                                save_best_only=True)


## Try with even more epochs

In [ ]:
# Train the model
history = model.fit(X_train, y_train, epochs=400, verbose=True, callbacks=cp_callback)

Epoch 1/400
858/858 [==============================] - 3s 3ms/step - loss: 0.5812 - accuracy: 0.7137
Epoch 2/400
858/858 [==============================] - 2s 2ms/step - loss: 0.5646 - accuracy: 0.7237
Epoch 3/400
858/858 [==============================] - 2s 2ms/step - loss: 0.5589 - accuracy: 0.7261
Epoch 4/400
858/858 [==============================] - 4s 5ms/step - loss: 0.5558 - accuracy: 0.7249
Epoch 5/400
858/858 [==============================] - 5s 5ms/step - loss: 0.5540 - accuracy: 0.7263
Epoch 6/400
858/858 [==============================] - 4s 4ms/step - loss: 0.5524 - accuracy: 0.7275
Epoch 7/400
858/858 [==============================] - 5s 5ms/step - loss: 0.5525 - accuracy: 0.7270
Epoch 8/400
858/858 [==============================] - 3s 3ms/step - loss: 0.5516 - accuracy: 0.7277
Epoch 9/400
858/858 [==============================] - 2s 2ms/step - loss: 0.5499 - accuracy: 0.7300
Epoch 10/400
858/858 [==============================] - 2s 2ms/step - loss: 0.5492 - accura

In [ ]:
test_loss, test_acc = model.evaluate(X_test, y_test)
print(f'Test accuracy: {test_acc}, Test loss: {test_loss}')

215/215 [==============================] - 1s 2ms/step - loss: 0.5673 - accuracy: 0.7354
Test accuracy: 0.7354227304458618, Test loss: 0.5673278570175171


# Save the model

In [ ]:
model.save('Models/charity_data_model_v6', save_format="h5")

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


# Load the model

In [ ]:
from tensorflow.keras.models import load_model

In [ ]:
loaded_model = load_model("Models/charity_data_model_v6")

In [ ]:
loaded_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 120)               5040      
                                                                 
 dense_1 (Dense)             (None, 100)               12100     
                                                                 
 dense_2 (Dense)             (None, 80)                8080      
                                                                 
 dense_3 (Dense)             (None, 1)                 81        
                                                                 
Total params: 25301 (98.83 KB)
Trainable params: 25301 (98.83 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
# Load the weights saved during training, should be the better performing
loaded_model.load_weights(checkpoint_path)

In [ ]:
# Best epoch weights version
# Check if the performance is actually better

test_loss, test_acc = loaded_model.evaluate(X_test, y_test)
print(f'Test accuracy: {test_acc}, Test loss: {test_loss}')

215/215 [==============================] - 0s 2ms/step - loss: 0.5669 - accuracy: 0.7350
Test accuracy: 0.7349854111671448, Test loss: 0.5668905973434448


In [ ]:
# Last epoch version
test_loss, test_acc = model.evaluate(X_test, y_test)
print(f'Test accuracy: {test_acc}, Test loss: {test_loss}')

215/215 [==============================] - 0s 2ms/step - loss: 0.5673 - accuracy: 0.7354
Test accuracy: 0.7354227304458618, Test loss: 0.5673278570175171


## The performance from the last epoch is better, this suggest overfitting, as the weights from the best epoch fail to improve performance with the test data